# Motivation

I'm curious it is possible to apply topic modeling to the chapters in the Bible in order to group similar chapters in some logical way, i.e. New Testament chapters, Psalms chapters, Minor Prophet chapters, etc. In order to do this I will first collapse all of the text in each chapter into a single observation. I will then process the chapter data and add the tokenized and clean list of words from each to a corpus. I will then apply various topic modeling algorithms to this corpus and analyze the results for the best fit. Once that's done, I will determine which chapters belong to the various topics. 

# Set up

This is my typical set up. I import the packages I will use, set my project directory, remove column and row limits, and allow Jupyter to display all of the output from each cell.

In [ ]:
import os
import pandas as pd
import numpy as np
import sqlite3
import spacy
from datetime import datetime

# Set project folder as directory
os.chdir(r'C:/Users/david/Projects/Bible Analytics')

# Remove row and column limits
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Display all output from each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Accessing data

In [ ]:
database = 'Data/SQL database.db'

In [ ]:
conn = sqlite3.connect(database)
 
df = pd.read_sql_query('SELECT * FROM t_web', conn)
 
conn.close

In [ ]:
df.info()
df.head()

# Begin

The first thing I will do is combine all of the text in each chapter into a single observation. This is actually quite simple. I will group our data by book and chapter and apply the sum function to our clean text column. Then I will reset the index so that book and chapter show up as columns. Finally, I will merge this data with out key dataset that contains the books names. Finally, I will reorder the columns to suit my personal preferences.

In [ ]:
chapter_text = pd.DataFrame(df.groupby(['b','c'])['clean_t'].sum())
chapter_text.reset_index(inplace=True)

key = pd.read_csv('Jupyter/Jupyter data/key_english.csv')

chapter_text = key.merge(chapter_text, how='inner', left_on='b', right_on='b')
chapter_text = chapter_text[['name', 'old_new', 'group', 'b', 'c', 'clean_t']]

chapter_text.head(2)

# Pushing chapter data to SQL database

Before moving forward, I want to save this dataset to our SQL dataset.

In [ ]:
conn = sqlite3.connect(database)

chapter_text.to_sql('chapter_text', conn, if_exists='replace', index=False)

conn.close()

In [ ]:
# *table* means double quotes around table
 
conn = sqlite3.connect(database)
cursor = conn.cursor()
 
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
 
for i in cursor.fetchall():
    print(i[0])
    
conn.close()

# Compiling stop words from NLP packages

Next, I will clean the data by removing stop words. There are several NLP packages that have their own lists of stop words. I've examined each of these lists and I'm comfortable removing the stop words contained in all of the list.

In [ ]:
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

from gensim.parsing.preprocessing import STOPWORDS
gen_stopwords = list(STOPWORDS)

nlp = spacy.load('en_core_web_lg')
spacy_stopwords = list(nlp.Defaults.stop_words)

print('There are', len(nltk_stopwords), 'stopwords in nltk,', len(gen_stopwords), 'in gensim, and', len(spacy_stopwords), 'in spacy.')

In [ ]:
stopwords = list(set(nltk_stopwords+gen_stopwords+spacy_stopwords+['shall', 'let', 'come', 'go', 'know', 'like']))
len(stopwords)

# Processing the text

I will also lemmatize the data after removing stopwords. Lemmatizing the text will revert each word back to its root. For instance, "run" will change to "ran." I will begin by defining an nlp object and loading the large, English language pipeline. I will then define an empty corpus list that we will use for topic modeling. There's some stuff about timing. Then, within a FOR loop I will iterate through the text for each chapter. First, I'll define an empty list called temp. Then, I will create a TRY block to handle any exceptions. Within this TRY block, I will create a document for each chapter by applying our nlp object to the chapter text. Then, I will created a nested FOR loop and iterate through each word of each chapter. Within this nested FOR loop, I first filter out any proper nouns. I've thought about this and decided I didn't want the topics to be driven by the people and places mentioned in each chapter. By removing proper nouns, I hope these topics will be more content driven. Next, I will use conditional statements to remove any word whose lemmatized version is in our customized list of stop words. For instance, "say" is in our list of stop words, so "said" would also be removed. I also removed punctuation. Then each lemmatized token is added to the list of words for that chapter. Once every token in a chapter has been iterated through, the entire list of lammetized words is added to our corpus list. Finally, an EXCEPT block is included to handle any errors and tell us where those errors occur so we can investigate.

In [ ]:
nlp = spacy.load("en_core_web_lg")

corpus_list = []

# Ignore this
start = datetime.now()
# Stop ignoring

for index, row in chapter_text.iterrows():
    
    temp=[]
    
    try:
        
        doc = nlp(row['clean_t'])        
    
        for token in doc:
            
            if token.pos_ != 'PROPN':
                
                # Removing stopwords
                if token.lemma_.lower() not in stopwords:
                    if not token.is_punct:
                        
                        temp+=[token.lemma_.lower()]
                    
        corpus_list.append(temp)
                
    except:
        
        print('Check out this chapter:')
        print(row['name'], row['c'])        
        print()

# Ignore this
stop = datetime.now()

print('This process took', stop-start)
print()


# Converting corpus_list into a corpus

Before I get started, I will first construct a dictionary that will map the words in the Bible to integer ids. I will then use this dictionary to convert each chapter of the Bible into a "bag of words" (bow). This collection of converted chapters is the corpus that I will run topic analysis on.

In [ ]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(corpus_list)

# Term Document Frequency
chapter_corpus = [id2word.doc2bow(text) for text in corpus_list]

# Modeling

Gensim provides several transformation models. Documentation for these models is here: https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html

# Term Frequency - Inverse Document Frequency

I will begin with TF-IDF. This transformation keeps the same vector space as the original corpus but down-weights commonly occurring terms while up-weighting less commonly occurring terms. I like this approach because it puts less emphasis on words that are consistently showing up throughout the corpus and more on the unique words that might differentiate our topics in a more interesting and meaningful way.

In [ ]:
from gensim import models

tfidf = models.TfidfModel(chapter_corpus)
tfidf_corpus = tfidf[chapter_corpus]

In [ ]:
print('Original word:',corpus_list[0][0])
print('Original weight:',chapter_corpus[0][0][1])
print('Transformed weight:',tfidf_corpus[0][0][1])

# Latent Semantic Indexing

Next, I would like to reduce dimensionality. Gensim offers several algorithms for this task, but I will focus on Latent Semantic Indexing (LSI). This algorithm will uncover the underlying patterns in the text and store that information a reduced vector space that we can then use to cluster the chapters in the Bible. The documentation recommends a target dimensionality of 200–500 for real-world data. I'll start with 200.

In [ ]:
import gensim
from pprint import pprint

# number of topics
num_topics = 200

# Build LSI model with original corpus
lsi_model_tfidf = gensim.models.LsiModel(corpus=tfidf_corpus,
                                            id2word=id2word,
                                            num_topics=num_topics)

pprint(lsi_model_tfidf.print_topics(2))

Each topic is a linear combination of the words in the text. We're only shown the ten most important terms for each topic, but in reality each topic is a linear combination of every term in the corpus. We can also see that that the first two topics share many of the same top-ten terms.

# Clustering LSI results

Next, I will use these topics to cluster. But first, we should remember that there are only 1,189 chapters in the Bible, so let's not go crazy. It wouldn't make sense to create 500 clusters with an average of 2.4 chapters in each. Instead, let's search for a number between 1 and 30. I'll create a topic matrix called X and then apply k-means clustering. I will then look at distortion and inertia to determine the correct number of topics.

In [ ]:
X = np.array([[tup[1] for tup in arr] for arr in lsi_model_tfidf[tfidf_corpus]])

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}

K = range(1,30)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
 
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
plt.plot(K, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

By examining both distortion and inertia, we see a clear "elbow" at 15 so this is the number of clusters we will start with.

In [ ]:
lsi_tfidf_15 = KMeans(n_clusters=15).fit(X)

# Examining cluster solution

I'm curious how many chapters are in each of our 15 clusters.

In [ ]:
labels = pd.DataFrame(lsi_tfidf_15.labels_, columns=['labels'])
len(labels)
labels.value_counts(['labels'])

This is interesting. We have several decent sized clusters that group together based on similar topics, but we also have a couple of really small clusters. Next, I will examine the chapters contained within each starting with the smallest cluster and working my way towards the largest. I hope to find underlying themes for each. First, I will label each chapter. Next, I will filter to each cluster and return a list of book names and chapters. I will then look up those chapters and try to understand what is connecting each cluster. If we've created good clusters, there should be shared themes.

In [ ]:
chapter_text['Labels'] = list(labels['labels'])

## Cluster 12 (Johnathan):

In [ ]:
len(chapter_text[chapter_text['Labels']==12])
chapter_text[chapter_text['Labels']==12][['name', 'c']]

Right way, I'm happy with this outcome. This cluster is very small but it picked up three chapters about Johnathan. What amazes me is that I removed all proper nouns from our corpus. This cluster is being formed on context alone and yet it still picked out three chapters about the same individual. That's pretty exciting.

## Cluster 14 (Interpretations):

In [ ]:
len(chapter_text[chapter_text['Labels']==14])
chapter_text[chapter_text['Labels']==14][['name', 'c']]

Cluster 14 has a very specific theme. Each chapter in this cluster is about the interpretation of various signs from the Lord, whether its dreams, writing on the wall, or even the stars themselves, in the case of the Wise Men.

## Cluster 8 (Genealogy):

In [ ]:
len(chapter_text[chapter_text['Labels']==8])
chapter_text[chapter_text['Labels']==8][['name', 'c']]

Cluster 8 is another very specific cluster. All thirty of these chapters contain genealogical records.

## Cluster 10 (Wisdom):

In [ ]:
len(chapter_text[chapter_text['Labels']==10])
chapter_text[chapter_text['Labels']==10][['name', 'c']]

This cluster contains 24 of the 31 chapters in Proverbs and nine of the 12 chapters in Ecclesiastes. In addition, we have the words of Elihu in Job 34 and six chapters in Psalms which all contain words of wisdom. It's interested that 33 of the 40 chapters in this clusters were written by Solomon.

## Cluster 3 (Laws and Regulations):

In [ ]:
len(chapter_text[chapter_text['Labels']==3])
chapter_text[chapter_text['Labels']==3][['name', 'c']]

There are 42 chapters in this cluster which all cover laws and regulations.

## Cluster 0 (Worship):

In [ ]:
len(chapter_text[chapter_text['Labels']==0])
chapter_text[chapter_text['Labels']==0][['name', 'c']]

Amazing! There are 48 chapters in this cluster and all but one come from Psalms. What fascinates me about this cluster is that these chapters all contain songs of worship. In fact, we still sing several worships songs that find their root in this list. Perhaps more fascinating, Psalm 22 is also in this list. This particular Psalm prophesies the death of Jesus in excruciating detail. To find this chapter in a cluster solely composed of worship songs is quite amazing!

## Cluster 13 (Hostel Kings):

In [ ]:
len(chapter_text[chapter_text['Labels']==13])
chapter_text[chapter_text['Labels']==13][['name', 'c']]

The chapters in this cluster are exclusively about dealing with hostel kings. All but the last chapter of Esther is in this cluster, Abram rescuing Lot from the five kings, Joshua facing several kings in battle, much of the conflict between Saul and David, Sennacherib threatening Jerusalem, Nebuchadnezzar subjugating Judah, and more. 

## Cluster 2 (The Royal Line):

In [ ]:
len(chapter_text[chapter_text['Labels']==2])
chapter_text[chapter_text['Labels']==2][['name', 'c']]

41 of the 51 chapters in this cluster come from Kings and Chronicles. Both document the royal lines of Judah and Israel. It also contains two chapters from 2 Samuel which are about the establishment of David's reign. The last chapter in this cluster is about the last king of Judah being taken into captivity. After this point in the Bible, there are no more kings in Israel.

## Cluster 4 (Building Instructions, censuses, and the Ark of the Covenant):

In [ ]:
len(chapter_text[chapter_text['Labels']==4])
chapter_text[chapter_text['Labels']==4][['name', 'c']]

This cluster includes chapters with very specific building instruction; the construction of Noah's ark, the tabernacle, Solomon's temple, and Nehemiah's dimensions of the temple. It also lists the results of several censuses and contains chapters that are primarily about the ark of the covenant. 

## Cluster 7 (The Gospels and Acts):

In [ ]:
len(chapter_text[chapter_text['Labels']==7])
chapter_text[chapter_text['Labels']==7][['name', 'c']]

This cluster includes almost all of the Gospels and Acts.

## Cluster 6 (Focused on the Church):

In [ ]:
len(chapter_text[chapter_text['Labels']==6])
chapter_text[chapter_text['Labels']==6][['name', 'c']]

I could not be any more thrilled with this solution. This cluster contains almost every chapter of every epistle except Hebrews. It also contains a few chapters from the Gospel that focus on the church after Jesus death and a chapter from Acts that establishes church law. 

## Cluster 9 (God's Direct Messages):

In [ ]:
len(chapter_text[chapter_text['Labels']==9])
chapter_text[chapter_text['Labels']==9][['name', 'c']]

Much of this cluster contain accounts of God speaking directly to His people, God's judgments, prophesied judgment and the division of the land. The golden calf, the Israelites refusing to enter the land, the bronze snake, Moses striking the rock. Almost all of Deuteronomy, which is Moses recounting the words of the Lord to the people of Israel. Most of Joshua is also in this cluster. Interestingly, Psalms 78 is in this cluster as well. This Psalm recounts the rebellions of Israel in the wilderness. And Psalms 105 which summarizes the history of Israel from Abraham to Moses. 

Almost all of Jeremiah and much of Ezekiel is in this cluster. Two major prophesies from Daniel about the Messiah and the Antichrist and about the fallout of Alexander the Greats kingdom are in this cluster as well. 

## Cluster 11 (History):

In [ ]:
len(chapter_text[chapter_text['Labels']==11])
chapter_text[chapter_text['Labels']==11][['name', 'c']]

This cluster contains the fall of man, Cain and Abel, and the entirety of Genesis from Abram to Joseph the the exception of three chapters and many other events from the Old Testament. In the New Testament, it contains the first two chapters of Luke which tell the story of Jesus prior to his ministry, Luke 15, which contains a few parables, Acts 7 in which Stephan recounts the history of the Israelites. With only a few excepts, this cluster seems to recount the history of God's people more so than any other cluster so far.

## Cluster 1 (God's Intervention):

In [ ]:
len(chapter_text[chapter_text['Labels']==1])
chapter_text[chapter_text['Labels']==1][['name', 'c']]

This cluster contains the creation of the world, God's covenant with Noah, God's covenant with Abram, the first nine plagues of Egypt, the splitting of the Red Sea, the waters of Marah, Mount Sinai, and God confirming his covenant with the people of Israel. Balaam blessing God's people and Moses blessing the tribes of Israel. Deborah's song about God's deliverance, David's song of praise in 2 Samuel, and many of the Psalms. Most of Isaiah and Amos, and a lot of the Old Testament prophets. Almost the entire book of Revelations. The theme of this cluster seems to be God's intervention or the great works of God. 

## Cluster 5 (Catch All):

In [ ]:
len(chapter_text[chapter_text['Labels']==5])
chapter_text[chapter_text['Labels']==5][['name', 'c']].reset_index(drop=True)

This cluster contains the majority of Job and just over half the chapters in Psalms. It also contains most of Jonah several chapters from the prophets. It also contains the story about making bricks with no straw, regulations about skin disease, discharge causing uncleanness, redeeming what is the Lords, budding of Aaron's staff, water of cleansing, making vows, towns for the Levis, God calling Samuel, and Nehemiah building the wall. From the New Testament, we have the Sermon on the Mount, some parables, several chapters from Hebrews and a few chapters from the various Epistles. 

Of all the clusters, this one seems the least interpretable. It really seems like a catch-all cluster for anything that doesn't fit nicely into the other 14. I'm not upset about this because there were bound to be chapters, even books that just didn't fit with all of the other themes in the Bible, but it would have been nice if this cluster had a clear theme of its own. Perhaps, if I had looked harder or been more familiar with the contents of these chapters a theme would have emerged.

# Conclusion

Many of the clusters have clearly defined themes and the chapters specific to books of the Bible often cluster together. All of the Gospels and Acts formed a cluster and the Epistles formed another. We have a cluster about genealogies and another about laws and regulations. There's even clusters about wisdom and songs of worships. Although not every cluster is so clearly themed, I am very happy with the outcome of this process.